# Cellpose Batch Segmentation

### **Authors:** oscardong4@gmail.com and heeva.baharlou@gmail.com (29/04/2024) - script adapted from [here](https://github.com/MouseLand/cellpose/blob/master/notebooks/run_cellpose_GPU.ipynb)

## Set your variables

In the code chunk below, alter the following variables:
- **model_path**: path to your custom model
- **analysis_dir**: path to your `analysis` folder
- **channels**: colour channels used for segmentation
- **cell_diameter**, **flow_threshold**, **cellprob_threshold**: other settings used when training your custom model

Remember to **run** the code chunk after setting the variables above. 

In [ ]:
# Set your required variables here
model_path = "IFMasksOnIMCModel_HumanColon_TN3_CD12_FT1"
analysis_dir = ""
channels = [2, 3] # This means Channel 1 was 'Green' and Channel 2 was 'Blue' (1 = R, 2 = G, 3 = B)
cell_diameter = 12.4
flow_threshold = 3
cellprob_threshold = -6

## Run the rest of the code
Start the batch segmentation by running the code chunk below. After running, you will notice **3 new** folders created in your `analysis` folder:
1. `3a_segmentation_masks`: contains the integer-labelled cell segmentation masks (1 for each image)
2. `3b_for_cellprofiler`: contains the segmentation masks **and** corresponding full stack images for use in CellProfiler
3. `5_cellprofiler_output`: an empty folder that will contain the output from CellProfiler 

After you see the message **'Done!'** printed, you can move to the next step on the GitHub page. 

In [1]:
# Import libraries
import os
import skimage.io
from cellpose import models, core
from cellpose.io import logger_setup
import shutil
from pathlib import Path

# Define Cellpose model
model = models.CellposeModel(gpu=core.use_gpu(), pretrained_model=model_path)
# Set and create directories
analysis = Path(analysis_dir)
image_dir = analysis / "2a_cellpose_full"
mask_dir = analysis / "3a_segmentation_masks"
os.makedirs(mask_dir, exist_ok=True)
# Call logger_setup to have output of cellpose written
logger_setup()
# Get list of image files
files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".tif")]  # Adjust the file extension if necessary
imgs = [skimage.io.imread(f) for f in files]
# Run segmentation
masks, flows, styles = model.eval(imgs, diameter=cell_diameter, flow_threshold=flow_threshold, cellprob_threshold=cellprob_threshold, channels=channels)
# Save mask images
for idx, mask in enumerate(masks):
    # Create a new file name by replacing the old directory with the new one
    file_name = files[idx].replace("cellpose", "segmentation_masks")
    # Add a suffix to indicate it's a mask
    file_name = os.path.splitext(file_name)[0] + "_mask.tif"
    skimage.io.imsave(file_name, mask)

# Create 'for_cellprofiler' directory 
full_path = analysis / "1c_full_images"
mask_path = analysis / "3a_segmentation_masks"
dest_path = analysis / "3b_for_cellprofiler"
dest_path.mkdir(exist_ok=True)
cellp_out = analysis / "5_cellprofiler_output"
cellp_out.mkdir(exist_ok=True)
# Copy relevant images to new directory
for file in (list(full_path.glob("*.tiff")) + list(mask_path.glob("*.tif"))):
    shutil.copy(file, dest_path)

print("Done!")